In [23]:
import pandas as pd
import os
import datetime
import locale
import re
import numpy as np
import matplotlib.pyplot as plt 

locale.setlocale(locale.LC_ALL, 'ko_kr')

'ko_kr'

# 데이터 준비

휴일여부 / 
교통량, 교통속도 /
추정유동인구 /
날씨 /
뉴스발행량, 뉴스조회

교통사고


In [24]:
# 교통량 데이터 로드
traffic = pd.read_csv('trf_grb.csv')
traffic.head(1)

,날짜,구,시간,traffic
0,20170101,강남구,0,27336


In [25]:
# 추정유동인구 데이터 로드 
flt = pd.read_csv('flt.csv')
flt.head(1)

,날짜,시간,구,추정유동인구
0,20170101,0,강남구,449000


In [26]:
# 날씨 데이터 로드
nalc = pd.read_csv('weather.csv')
nalc.head(1)

,날짜,시간,시정,싸락우박,우박,싸락눈,비,소낙비,연무,이슬비,소낙눈,박무,진눈깨비,안개,황사,맑음,눈
0,20170101,0,491,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# 교통사고 데이터 로드
acid4 = pd.read_csv('acid_case4.csv')
acid_event = pd.read_csv('acid_event.csv')

In [28]:
# 교통속도 데이터 로드
velocity = pd.read_csv('vel_melt.csv')
velocity.head(1)

,날짜,구,시간,교통속도
0,20170101,강남구,0,39.896499


In [29]:
# 휴일 정보
holiday = pd.read_csv('holiday.csv')
holiday['주말'] = 1
holiday = holiday[['locdate','주말']]
holiday.columns = ['날짜','주말']
holiday = holiday.drop_duplicates()
holiday.head(1)

# ohe_merge['요일'] = ohe_merge['날짜'].apply(lambda x: datetime.date(int(x[:4]),int(x[4:6]),int(x[6:])).strftime('%A')[0])
# ohe_merge = pd.merge(ohe_merge, year_merge, how='left', on = '날짜')
# ohe_merge.loc[(ohe_merge['요일'] == '일') |  (ohe_merge['요일'] =='토'),'주말'] = 1
# ohe_merge['주말'] = ohe_merge['주말'].fillna(0)

,날짜,주말
0,20170101,1


In [30]:
# 뉴스 발행량 데이터
news_vol = pd.read_csv('news_issue.csv')
news_vol_m = pd.read_csv('news_issue_1month.csv')
news_vol_w = pd.read_csv('news_issue_1week.csv')

# 뉴스 조회지수 데이터
news_views = pd.read_csv('scaled_views.csv')
news_views_m = pd.read_csv('scaled_views_month.csv')
news_views_w = pd.read_csv('scaled_views_week.csv')

news_views.head(1)


,날짜,조회지수
0,20170101,1.11467


In [31]:
news_vol_w.head()

,날짜,뉴스발행량
0,20170101,13
1,20170102,83
2,20170103,34
3,20170104,29
4,20170105,31


In [32]:
news_vol

,날짜,뉴스발행량
0,20170101,0
1,20170102,0
2,20170103,1
3,20170104,0
4,20170105,2
...,...,...
1090,20191227,0
1091,20191228,0
1092,20191229,0
1093,20191230,2


In [33]:
# news_vol.columns = ['날짜','뉴스발행량']
# news_vol_m.columns = ['날짜','뉴스발행량']
# news_vol_w.columns = ['날짜','뉴스발행량']
# news_views.columns = ['날짜','조회지수']
# news_views_m.columns = ['날짜','조회지수']
# news_views_w.columns = ['날짜','조회지수']

# news_views_m = news_views_m[:1095]

# news_vol['날짜'] = news_vol['날짜'].apply(lambda x: str(x).replace('-',''))
# news_vol_m['날짜'] = news_vol_m['날짜'].apply(lambda x: str(x).replace('-',''))
# news_vol_w['날짜'] = news_vol_w['날짜'].apply(lambda x: str(x).replace('-',''))
# news_views['날짜'] = news_views['날짜'].apply(lambda x: str(x).replace('-',''))
# news_views_m['날짜'] = news_views_m['날짜'].apply(lambda x: str(x).replace('-',''))
# news_views_w['날짜'] = news_views_w['날짜'].apply(lambda x: str(x).replace('-',''))

# news_vol.to_csv('news_issue.csv', index=False, encoding = 'utf-8')
# news_vol_m.to_csv('news_issue_1month.csv', index=False, encoding = 'utf-8')
# news_vol_w.to_csv('news_issue_1week.csv', index=False, encoding = 'utf-8')
# news_views.to_csv('scaled_views.csv', index=False, encoding = 'utf-8')
# news_views_m.to_csv('scaled_views_month.csv', index=False, encoding = 'utf-8')
# news_views_w.to_csv('scaled_views_week.csv', index=False, encoding = 'utf-8')

In [34]:
# 종속변수 추가
acid4 = pd.read_csv('acid_case4.csv')
acid_event = pd.read_csv('acid_event.csv')

acid_event

,날짜,시간,구,건수
0,20170101,0,강북구,1
1,20170101,0,노원구,1
2,20170101,0,동작구,1
3,20170101,0,서초구,1
4,20170101,0,송파구,1
...,...,...,...,...
103541,20191231,22,동작구,1
103542,20191231,22,송파구,1
103543,20191231,22,영등포구,1
103544,20191231,22,중랑구,1


# 테이블 merge

휴일여부 / 교통량, 교통속도 / 추정유동인구 / 날씨 / 뉴스발행량 월,주 / 뉴스조회 월, 주      
holiday / traffic, velocity / flt / nalc / news_vol_m news_vol_w / news_views_m news_views_w

교통사고     
acid4 acid_event

key 
'날짜', '시간', '구'

3년간 시간별 구별 데이터    
3 * 365 * 24 * 24 = 630720 행

In [35]:
dataset = pd.merge(traffic, velocity, on = ['날짜', '시간', '구'], how = 'left')
dataset = pd.merge(dataset, flt, on = ['날짜', '시간', '구'], how = 'left')
dataset = pd.merge(dataset, nalc, on = ['날짜', '시간'], how = 'left')
dataset = pd.merge(dataset, holiday, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, news_views_m, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, news_vol_m, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, acid_event, on = ['날짜', '시간', '구'], how = 'left')
dataset = dataset.sort_values(['날짜','구','시간'])

dataset = dataset.fillna(0)
dataset['날짜'] = dataset['날짜'].apply(str)
dataset['요일'] = dataset['날짜'].apply(lambda x: datetime.date(int(x[:4]),int(x[4:6]),int(x[6:])).strftime('%A')[0])
dataset.loc[(dataset['요일'] == '일') |  (dataset['요일'] =='토'),'주말'] = 1
dataset = dataset.drop('요일',axis=1)

dataset.to_csv('dataset_month.csv', index = False, encoding='utf-8')

In [36]:
dataset = pd.merge(traffic, velocity, on = ['날짜', '시간', '구'], how = 'left')
dataset = pd.merge(dataset, flt, on = ['날짜', '시간', '구'], how = 'left')
dataset = pd.merge(dataset, nalc, on = ['날짜', '시간'], how = 'left')
dataset = pd.merge(dataset, holiday, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, news_views_w, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, news_vol_w, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, acid_event, on = ['날짜', '시간', '구'], how = 'left')
dataset = dataset.sort_values(['날짜','구','시간'])

dataset = dataset.fillna(0)
dataset['날짜'] = dataset['날짜'].apply(str)
dataset['요일'] = dataset['날짜'].apply(lambda x: datetime.date(int(x[:4]),int(x[4:6]),int(x[6:])).strftime('%A')[0])
dataset.loc[(dataset['요일'] == '일') |  (dataset['요일'] =='토'),'주말'] = 1
dataset = dataset.drop('요일',axis=1)

dataset.to_csv('dataset_week.csv', index = False, encoding='utf-8')

In [13]:
dataset = pd.merge(traffic, velocity, on = ['날짜', '시간', '구'], how = 'left')
dataset = pd.merge(dataset, flt, on = ['날짜', '시간', '구'], how = 'left')
dataset = pd.merge(dataset, nalc, on = ['날짜', '시간'], how = 'left')
dataset = pd.merge(dataset, holiday, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, news_views, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, news_vol_w, on = ['날짜'], how = 'left')
dataset = pd.merge(dataset, acid_event, on = ['날짜', '시간', '구'], how = 'left')
dataset = dataset.sort_values(['날짜','구','시간'])

dataset = dataset.fillna(0)
dataset['날짜'] = dataset['날짜'].apply(str)
dataset['요일'] = dataset['날짜'].apply(lambda x: datetime.date(int(x[:4]),int(x[4:6]),int(x[6:])).strftime('%A')[0])
dataset.loc[(dataset['요일'] == '일') |  (dataset['요일'] =='토'),'주말'] = 1
dataset = dataset.drop('요일',axis=1)

dataset.to_csv('dataset.csv', index = False, encoding='utf-8')

In [42]:
df_for_visual = dataset.loc[:, ['날짜', '구' ,'시간', 'traffic', '교통속도', '추정유동인구']]

In [43]:
df_for_visual.head()

,날짜,구,시간,traffic,교통속도,추정유동인구
0,20170101,강남구,0,27336,39.896499,449000
11,20170101,강남구,1,27429,40.277729,441924
16,20170101,강남구,2,18495,42.811063,436549
17,20170101,강남구,3,15075,44.266713,431175
18,20170101,강남구,4,13356,44.933735,425355
